# Logic

In [1]:
from matplotlib import image
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp "/content/drive/MyDrive/AML_project/list/train.txt" "train.txt"

In [6]:
# Extracting the labels from the train.txt for encoding and decoding purposes
with open('train.txt') as txt_file:
  lines_arr = [line.strip() for line in txt_file.readlines()]
  labels = [line.split(' ')[1] for line in lines_arr]

In [7]:
# As we have encoded the labels on the training process, we might need to also have the decoder to change the encoded labels
# back to its true form of species codes thus it is easier to find 
# the real name of the species later on
def decode_labels(all_labels, target_label):
  label_encoder = LabelEncoder()
  label_encoder.fit(all_labels)
  return label_encoder.inverse_transform(target_label)  

# GUI

In [8]:
!pip install pyngrok==4.1.1
!pip install flask-ngrok
!pip install flask-bootstrap
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15982 sha256=83d600ffd9cff465901a7c983eec310ee56bd8387f453c34016a7d7c5528cdeb
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 456 kB 6.6 MB/s 
  Created wheel for flask-bootstrap: filename=Flask_Bootstrap-3.3.7.1-py3-none-any.whl size=460123 sha256=7bd5445b8175416164d7f08c343d6a0b42cce1c46d1596a5e878df3473dbbdda
  Stored in directory: /root/.cache/pip/wheels/67/a2/d6/50d039c9b59b4caca6d7b53839c8100354a52ab7553d2456eb
  Created wheel for visitor: filename=visitor-0.1.3-py3-none-any.whl size=3943 sha

In [9]:
!ngrok authtoken 2A700gdUVgeF9pUCudFiGfb5Nu1_6ED94twFJEyKzM3dFZhQE

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [10]:
# Extracting list of species from species.txt 
# for mapping the predicted value into the species value
def load_species(text_file):
  with open(text_file) as txt_file:
    species_dict = {}
    for line in txt_file.readlines():
      split_line = line.split(';')
      class_value = split_line[0]
      species_name = split_line[1].strip()

      species_dict[class_value] = species_name
    return species_dict


In [11]:
import os
from PIL import Image
import os.path
import io
import base64

# Returning the Herbarium from the training set 
# To show the references 
# Currently return up to 5 Herbarium references
def load_images_from_folder(folder):
    images = []
    counter = 0
    for filename in os.listdir(folder):
        image = os.path.join(folder,filename)
        im = Image.open(image)
        data = io.BytesIO()
        im.save(data, "JPEG")

        encoded_img_data = base64.b64encode(data.getvalue())
        images.append(encoded_img_data.decode('utf-8'))
        counter += 1

        if counter == 5:
          return images

In [12]:
from matplotlib import image

# To preprocess the input image from the user
def preprocess_image(image_file):
    image_data = image.imread(image_file)
    image_data = tf.cast(image_data, tf.float32)/255
    image_data = tf.image.resize(image_data, (128, 128), method = "bilinear")
    image_data = np.array(image_data)
    image_data = image_data.ravel()
    image_data = image_data.reshape(1, 128, 128, 3)

    return image_data

In [13]:
from tensorflow import keras
model = tf.keras.models.load_model('drive/MyDrive/AML_project/saved_models/', compile=False)

def model_predict(image_data):
  y_pred_score = model.predict(image_data)
  y_pred = np.argmax(y_pred_score, axis=1)
  return decode_labels(labels, y_pred)[0]

In [14]:
species_dict = load_species("/content/drive/MyDrive/AML_project/list/species_list.txt")
species_dict

{'105951': 'Maripa glabra Choisy',
 '106023': 'Merremia umbellata (L.) Hallier f.',
 '106387': 'Costus arabicus L.',
 '106461': 'Costus scaber Ruiz Pav.',
 '106466': 'Costus spiralis (Jacq.) Roscoe',
 '110432': 'Evodianthus funifer (Poit.) Lindm.',
 '116853': 'Pteridium arachnoideum (Kaulf.) Maxon',
 '119986': 'Olfersia cervina (L.) Kunze',
 '120497': 'Diospyros capreifolia Mart. ex Hiern',
 '121836': 'Sloanea grandiflora Sm.',
 '121841': 'Sloanea guianensis (Aubl.) Benth.',
 '12254': 'Anacardium occidentale L.',
 '12518': 'Mangifera indica L.',
 '125412': 'Sphyrospermum cordifolium Benth.',
 '126895': 'Syngonanthus caulescens (Poir.) Ruhland',
 '127007': 'Tonina fluviatilis Aubl.',
 '127097': 'Erythroxylum fimbriatum Peyr.',
 '127151': 'Erythroxylum macrophyllum Cav.',
 '127242': 'Erythroxylum squamatum Sw.',
 '12910': 'Spondias mombin L.',
 '12922': 'Tapirira guianensis Aubl.',
 '129645': 'Croton schiedeanus Schltdl.',
 '130657': 'Euphorbia cotinifolia L.',
 '131079': 'Euphorbia hete

In [15]:
!unzip drive/MyDrive/AML_project/train.zip > /dev/null
!unzip drive/MyDrive/AML_project/test.zip > /dev/null

In [ ]:
import base64
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request, redirect, url_for

# import the source codes to Flask and host it on ngrok
app = Flask(__name__, template_folder = './drive/MyDrive/AML_project/GUI/templates', static_folder = './drive/MyDrive/AML_project/GUI/static')
run_with_ngrok(app)

@app.route("/")
def index():
  return render_template("index.html")

@app.route("/image", methods=['POST'])
def my_form_post():
  image_upload = request.files['file']
  image_data = preprocess_image(image_upload)
  pred = model_predict(image_data)

  image_b64 = base64.b64encode(image_upload.getvalue()).decode('utf-8')

  prediction = str(pred)

  prediction_text = species_dict[prediction]

  images = load_images_from_folder("train/herbarium/{}".format(prediction))
  
  return render_template("showImage.html", image_name=image_upload.filename, image_list=images, image_upload=image_b64, prediction=prediction_text)


# Run the Flask Web App
if __name__ == '__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://296c-35-247-15-146.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [26/Nov/2022 17:17:18] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2022 17:17:18] "GET /static/index.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2022 17:17:19] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2022 17:17:20] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2022 17:17:21] "GET /static/index.css HTTP/1.1" 200 -


1/1 [==============================] - 0s 20ms/step


INFO:werkzeug:127.0.0.1 - - [26/Nov/2022 17:17:25] "POST /image HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2022 17:17:25] "GET /image HTTP/1.1" 405 -


1/1 [==============================] - 0s 21ms/step


INFO:werkzeug:127.0.0.1 - - [26/Nov/2022 17:17:30] "POST /image HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2022 17:17:32] "GET /static/assets/arrow.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2022 17:17:32] "GET /favicon.ico HTTP/1.1" 404 -
